<a href="https://colab.research.google.com/github/xuanyu410/114-1PL-Repo/blob/main/%E7%A8%8B%E5%BC%8F%E8%AA%9E%E8%A8%80%E4%BD%9C%E6%A5%AD%E4%BA%8C.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [29]:
!pip install -q gradio gspread pandas matplotlib pillow google-generativeai

In [30]:
# === Google 驗證 ===
from google.colab import auth
auth.authenticate_user()

import gspread
from google.auth import default
import pandas as pd
from datetime import datetime
import gradio as gr
import matplotlib.pyplot as plt
from io import BytesIO
from PIL import Image
import google.generativeai as genai
from google.colab import userdata

In [31]:
# === Gemini AI 設定 ===
api_key = userdata.get("repo")   # 在 Colab > 設定 > Secrects 放入 key
genai.configure(api_key=api_key)
gemini_model = genai.GenerativeModel("gemini-1.5-flash")

In [32]:
# === 連線到試算表 ===
creds, _ = default()
gc = gspread.authorize(creds)

SPREADSHEET_URL = "https://docs.google.com/spreadsheets/d/1UIfts0iHJzLn6VdOeuT3WS7UKDEdS5dylr9WxK1BhFA/edit?gid=0#gid=0"
ss = gc.open_by_url(SPREADSHEET_URL)
worksheet = ss.worksheet("工作表1")

# === 確保標題列 ===
headers = ["日期", "時間", "品項", "種類", "金額"]
if worksheet.row_values(1) != headers:
    worksheet.clear()
    worksheet.append_row(headers, value_input_option="RAW")

In [33]:
# === 工具函式 ===
def _read_sheet_to_df():
    rows = worksheet.get_all_values()
    if len(rows) <= 1:
        return pd.DataFrame(columns=headers)
    df = pd.DataFrame(rows[1:], columns=rows[0])
    df = df.loc[:, ~df.columns.duplicated()].copy()
    df["金額"] = pd.to_numeric(df["金額"].replace(r"[^\d.\-]+", "", regex=True), errors="coerce").fillna(0.0)
    df["日期_dt"] = pd.to_datetime(df["日期"], errors="coerce")
    df["日期_str"] = df["日期_dt"].dt.strftime("%Y-%m-%d")
    return df

def _make_pie_image(summary, title=""):
    if summary.empty: return None
    fig, ax = plt.subplots(figsize=(5,5))
    ax.pie(summary["金額"], labels=summary["種類"], autopct="%1.1f%%", startangle=90)
    ax.set_title(title)
    buf = BytesIO()
    plt.tight_layout()
    fig.savefig(buf, format="png")
    plt.close(fig)
    buf.seek(0)
    return Image.open(buf).convert("RGB")

In [34]:
# === 寫入支出 ===
def add_expense(date_str, time_str, item, category, amount):
    try:
        amount_val = float(amount)
    except:
        return "❌ 金額必須是數字", None

    if not date_str:
        date_str = datetime.now().strftime("%Y-%m-%d")
    if not time_str:
        time_str = datetime.now().strftime("%H:%M")

    worksheet.append_row([date_str, time_str, item, category, amount_val], value_input_option="RAW")
    return f"✅ 已新增：{date_str} | {time_str} | {item} | {category} | {amount_val}", None

# === 查詢每日總支出 ===
def show_today_total(date_str):
    df = _read_sheet_to_df()
    if df.empty: return "無資料", None
    target = df[df["日期_str"] == date_str]
    if target.empty: return f"{date_str} 無資料", None
    total = target["金額"].sum()
    cat_summary = target.groupby("種類", as_index=False)["金額"].sum()
    img = _make_pie_image(cat_summary, f"{date_str} 各種類支出")
    msg = f"📅 {date_str} 總支出：{int(total)} 元\n\n{cat_summary.to_string(index=False)}"
    return msg, img

# === 查詢每月總支出 ===
def show_month_total(month_str):
    try:
        year, month = map(int, month_str.split("-"))
    except:
        return "❌ 輸入格式錯誤 (需 YYYY-MM)", None
    df = _read_sheet_to_df()
    if df.empty: return "無資料", None
    target = df[(df["日期_dt"].dt.year == year) & (df["日期_dt"].dt.month == month)]
    if target.empty: return f"{month_str} 無資料", None
    total = target["金額"].sum()
    cat_summary = target.groupby("種類", as_index=False)["金額"].sum()
    img = _make_pie_image(cat_summary, f"{month_str} 各種類支出")
    msg = f"🗓️ {month_str} 總支出：{int(total)} 元\n\n{cat_summary.to_string(index=False)}"
    return msg, img

# === 日期時間按鈕 ===
def fill_today(): return datetime.now().strftime("%Y-%m-%d")
def fill_nowtime(): return datetime.now().strftime("%H:%M")


In [35]:
# === AI 分析支出 ===
def ai_analyze_expenses():
    df = _read_sheet_to_df()
    if df.empty:
        return "目前沒有任何支出紀錄。"
    # 轉成文字表格給 AI
    records = df.tail(30).to_string(index=False)  # 最近30筆
    prompt = f"以下是最近的消費紀錄，請分析我的花費習慣，指出主要花在哪些類別，並提供省錢建議：\n\n{records}"
    response = gemini_model.generate_content(prompt)
    return response.text

In [41]:
# === Gradio UI ===
categories = ["早餐", "午餐", "晚餐", "點心", "飲料", "衣服", "娛樂", "交通"]

with gr.Blocks() as demo:
    gr.Markdown("## 📝 個人記帳系統 + AI 分析")

    with gr.Tab("新增支出"):
        with gr.Row():
            date_in = gr.Textbox(label="日期 (YYYY-MM-DD)")
            btn_today = gr.Button("📅 今天")
            time_in = gr.Textbox(label="時間 (HH:MM)")
            btn_now = gr.Button("⏰ 現在時間")
        item_in = gr.Textbox(label="品項")
        amount_in = gr.Textbox(label="金額")
        cat_in = gr.Dropdown(choices=categories, label="種類", value="午餐")
        submit_btn = gr.Button("✅ 新增記錄")
        out_text = gr.Textbox(label="結果")
        out_img = gr.Image(label="回饋圖", visible=False)  # 預留

        btn_today.click(fn=fill_today, inputs=None, outputs=date_in)
        btn_now.click(fn=fill_nowtime, inputs=None, outputs=time_in)
        submit_btn.click(add_expense, [date_in,time_in,item_in,cat_in,amount_in], [out_text,out_img])

    with gr.Tab("查詢當日總支出"):
        date_q = gr.Textbox(label="日期 (YYYY-MM-DD)")
        btn_day = gr.Button("📅 查詢當日總支出")
        day_out = gr.Textbox(label="結果")
        day_img = gr.Image(label="圓餅圖")
        btn_day.click(show_today_total, [date_q], [day_out, day_img])

    with gr.Tab("查詢當月總支出"):
        month_q = gr.Textbox(label="月份 (YYYY-MM)", placeholder="例如 2025-09")
        btn_month = gr.Button("🗓️ 查詢當月總支出")
        month_out = gr.Textbox(label="結果")
        month_img = gr.Image(label="圓餅圖")
        btn_month.click(show_month_total, [month_q], [month_out, month_img])

    with gr.Tab("AI 消費分析"):
        btn_ai = gr.Button("🤖 生成 AI 消費分析")
        ai_out = gr.Textbox(label="AI 分析結果", lines=10)
        btn_ai.click(ai_analyze_expenses, None, ai_out)

demo.launch(share=True)

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://d3e1c68dcdfff03d7a.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
